In [3]:
from PIL import Image
from bs4 import BeautifulSoup
from IPython.display import display, clear_output
import random
import requests
import time
import json


data_consent_info = """DATA CONSENT INFORMATION:

Please read:

we wish to record your response data

to an anonymised public data repository.

Your data will be used for educational teaching purposes

practising data analysis and visualisation.

Please type yes in the box below if you consent to the upload."""

print(data_consent_info)

result = input("> ")

if result == "yes":
    print("Thanks for your participation.")
    print("Please contact philip.lewis@ucl.ac.uk")
    print("If you have any questions or concerns")
    print("regarding the stored results.")

else:

# end code execution by raising an exception

    raise(Exception("User did not consent to continue test."))

clear_output()

id_instructions = """

Enter your anonymised ID

To generate an anonymous 4-letter unique user identifier please enter:

- two letters based on the initials (first and last name) of a childhood friend

- two letters based on the initials (first and last name) of a favourite actor / actress

e.g. if your friend was called Charlie Brown and film star was Tom Cruise

then your unique identifer would be CBTC

"""

print(id_instructions)

user_id = input("> ")

print("User entered id:", user_id)

clear_output()

#send to google form
def send_to_google_form(data_dict, form_url):
    ''' Helper function to upload information to a corresponding Google Form 
        You are not expected to follow the code within this function!
    '''
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')

    if content is None:
        print("Script tag not found. Data not uploaded.")
        return False

    content_text = content.text[27:-1] if content.text else None

    if not content_text:
        print("Content text not found. Data not uploaded.")
        return False

    try:
        result = json.loads(content_text)[1][1]
        print("Content of the script tag:")
        print(result)
        form_dict = {}
    
        loaded_all = True
        for item in result:
            # Remove non-alphanumeric characters from the item name
            item_name_cleaned = ''.join(char.lower() for char in item[1] if char.isalnum())
            
            # Check if the cleaned item name matches any key in the data dictionary
            matching_keys = [key for key in data_dict.keys() if item_name_cleaned in ''.join(char.lower() for char in key if char.isalnum())]
            
            if not matching_keys:
                print(f"Form item {item[1]} not found. Data not uploaded.")
                loaded_all = False
                return False
            
            # Use the first matching key as the data dictionary key
            form_dict[f'entry.{item[4][0][0]}'] = data_dict[matching_keys[0]]
        
        post_result = requests.post(post_form_url, data=form_dict)
        return post_result.ok
    
    except (json.JSONDecodeError, IndexError) as e:
        print(f"Error decoding JSON or accessing result: {e}")
        return False

print("Hello! What is your name?")
user_name = input("> ")

print("How old are you?")
age = input("> ")

print("What is your gender?")
gender = input("> ")


clear_output()

results = []

form_url = "https://docs.google.com/forms/d/e/1FAIpQLSfI7JKtNgG8hwV3ypkuXPxx-7W75r2bbgWOjsm6rsx8aI4LCw/viewform?usp=sf_link"

        
class ImageDisplay:
    def __init__(self, image_path, duration, callback):
        self.img = Image.open(image_path).convert('RGB')
        display(self.img)

        time.sleep(duration)
        clear_output()
        callback()

def ask_question(question, answer):
    user_input = input(question)

    if user_input and user_input.lower() == str(answer).lower():
        print("Great!")
        return True
    else:
        print("Incorrect!")
        return False


def memory_test():
    random.seed(1) 

    introduction = f"{user_name}, you will be shown a grid of 9 different shapes of 9 colors."
    print(introduction)

    image_path = "shapes grid.png"
    def after_image_disappears():
        ask_questions(user_name, user_id, data_dict)

    display_obj = ImageDisplay(image_path, 10, after_image_disappears)


def ask_questions(user_name, user_id, data_dict):
    global score
    

    questions = [
        ("Which shape was pink?", "star"),
        ("What color was the triangle?", "green"),
        ("Which row was the pentagon in? (1, 2, or 3)", "2"),
        ("What color was the hexagon?", "black"),
        ("Was there an orange shape? (yes or no)", "no")]

    for question, answer in questions:
        if ask_question(question, answer):
            score += 1

    clear_output()
    print(f"Your Score: {score} out of {len(questions)}. ")

    return score

score=0
global score

data_dict={
    'User ID':user_id, 
    'Name': user_name, 
    'Age': age,
    'Gender':gender, 
    'Score':score
    }

if __name__ == "__main__":
    memory_test()

data_dict={
    'User ID':user_id, 
    'Name': user_name, 
    'Age': age,
    'Gender':gender, 
    'Score':score
    }

upload_success = send_to_google_form(data_dict, form_url)

# Print the result
if upload_success:
    print("Data uploaded successfully!")
else:
    print("Data upload failed.")

Your Score: 1 out of 5. 
Content of the script tag:
[[1503898043, 'Name', None, 0, [[803561305, None, 0]], None, None, None, None, None, None, [None, 'Name']], [186914487, 'ID', None, 0, [[1685699984, None, 0]], None, None, None, None, None, None, [None, 'ID']], [1624665587, 'Age', None, 0, [[316858222, None, 0]], None, None, None, None, None, None, [None, 'Age']], [1886762185, 'Gender', None, 0, [[1403501342, None, 0]], None, None, None, None, None, None, [None, 'Gender']], [348676824, 'Score', None, 0, [[1567589063, None, 0]], None, None, None, None, None, None, [None, 'Score']]]
Data uploaded successfully!


In [13]:
print(score)

2
